In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dates_and_times').getOrCreate()

23/12/13 20:58:00 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.19.51.88 instead (on interface eth0)
23/12/13 20:58:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 20:58:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/13 20:58:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/13 20:58:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [5]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [7]:
from pyspark.sql.functions import dayofmonth, dayofweek, dayofyear, hour,month,year,weekofyear,format_number,date_format

In [12]:
df.select(dayofmonth('Date')).show()
df.select(dayofweek('Date')).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows

+---------------+
|dayofweek(Date)|
+---------------+
|              2|
|              3|
|              4|
|              5|
|              6|
|              2|
|              3|
|              4|
|              5|
|              6|
|              3|
|              4|
|              5|
|              6|
|              2|
|              3|
|              4|
|              5|
|              6|
|              2|
+---------------+


In [14]:
df.select(year('Date')).show()
df.select(month('Date')).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+


In [31]:
new_df = df.withColumn("Year", year(df['Date']))

In [36]:
new_df.groupBy('Year').sum('Open', 'Close').show()

+----+------------------+------------------+
|Year|         sum(Open)|        sum(Close)|
+----+------------------+------------------+
|2015| 30284.28999099999|30250.079950999978|
|2013|119228.29016200003|     119103.989832|
|2014| 74375.94012300001| 74441.39009599996|
|2012|     144163.180197|144012.42989100004|
|2016| 26335.95998699999|26360.209983000004|
|2010| 65509.32001300001| 65480.29992000004|
|2011| 91743.47978900002| 91729.08998100003|
+----+------------------+------------------+


In [45]:
result = new_df.groupBy('Year').mean().select(["Year", "avg(Open)","avg(Close)"])
result.show()

+----+------------------+------------------+
|Year|         avg(Open)|        avg(Close)|
+----+------------------+------------------+
|2015|120.17575393253965|120.03999980555547|
|2013| 473.1281355634922| 472.6348802857143|
|2014| 295.1426195357143| 295.4023416507935|
|2012|     576.652720788| 576.0497195640002|
|2016|104.50777772619044|104.60400786904763|
|2010| 259.9576190992064| 259.8424600000002|
|2011|364.06142773412705|364.00432532142867|
+----+------------------+------------------+


In [46]:
from pyspark.sql.functions import format_number

In [58]:
result.select(['Year',format_number('avg(Open)', 2).alias('Open'),format_number('avg(Close)', 2).alias('Close') ]).orderBy(result['avg(Close)'].desc() ).show()

+----+------+------+
|Year|  Open| Close|
+----+------+------+
|2012|576.65|576.05|
|2013|473.13|472.63|
|2011|364.06|364.00|
|2014|295.14|295.40|
|2010|259.96|259.84|
|2015|120.18|120.04|
|2016|104.51|104.60|
+----+------+------+
